In [ ]:
import sys, os
sys.path.append("..")
os.chdir('/Users/johnnyxcy/Workspace/OpenSource/PyTrail/pytrial')

In [ ]:
from pytrial.data.demo_data import load_mimic_ehr_sequence, load_trial_outcome_data

In [ ]:
from pytrial.tasks.trial_patient_match.data import TrialData, PatientData

In [ ]:
df = load_trial_outcome_data()['data']

In [ ]:
df.columns

In [ ]:
df = df.iloc[:10] # make subsampling
trial_data = TrialData(df,encode_ec=True)

In [ ]:
df

In [ ]:
# build patient data class

# load demo ehr sequence
ehr = load_mimic_ehr_sequence(n_sample=1000)

# we first simulate the eligibility criteria matching labels for each patient
num_inc_ec = len(trial_data.inc_vocab)
num_exc_ec = len(trial_data.exc_vocab)
ec_label_list = []
for i in range(len(ehr['feature'])):
    # randomly choose a trial that this patient satisfies
    trial = df.sample(1)
    ec_label_list.append([trial['inclusion_criteria_index'].values[0], trial['exclusion_criteria_index'].values[0]])

In [ ]:
# build patient seq data
# ec_label_list, first is matched inclusion criteria, second is matched exclusion criteria
ehr_data = PatientData(
    data={'v':ehr['visit'], 'y': ec_label_list,  'x':ehr['feature'],},
    metadata={
        'visit': {'mode':'dense', 'order':ehr['order']},
        'label': {'mode':'dense'},
        'voc': ehr['voc'],
        },
)

In [ ]:
ehr_data

In [ ]:
from pytrial.tasks.trial_patient_match import DeepEnroll

In [ ]:
model = DeepEnroll(
    vocab_size=[len(ehr["voc"][o]) for o in ehr["order"]],
    order=ehr["order"],
    epochs=5,
    batch_size=128,
    device="cpu",
)

In [ ]:
model.fit(
    train_data={'patient':ehr_data, 'trial':trial_data},
    valid_data={'patient':ehr_data, 'trial':trial_data},
)

In [ ]:
# make predictions
res = model.predict(test_data={
    'patient': ehr_data,
    'trial': trial_data,
    }, return_dict=True)

In [ ]:
res["pred_trial"].keys()

In [ ]:
df_test = load_trial_outcome_data(split="test")['data']
trial_data_test = TrialData(df_test,encode_ec=True)

In [ ]:
res_test = model.predict(test_data={
    'patient': ehr_data,
    'trial': trial_data_test,
}, return_dict=True)

In [ ]:
import numpy as np

In [ ]:
ehr_data.metadata

In [ ]:
len(res_test['pred_trial']["NCT01288573"])

In [ ]:
for k in res["pred_trial"].keys():
    print(sum(res["pred_trial"][k]))